In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv("/content/online_retail_II.csv", encoding="ISO-8859-1")

In [ ]:
df.head(15)

# Recency (Yenilik)
Müşteri en son ne zaman alışveriş yaptı ?
# Frequency (Sıklık)
Müşteri toplamda kaç kez alışveriş yaptı ?
# Monetary (Parasal değer)
Müşteri toplamda ne kadar para harcadı ?


Amacımız bu beş metriğe beş üzerinden bir puan vermek.
Her müşteri Id'si için bu metriklerin sayısını tanımlamam gerekiyor.
Sonra kararlaştırdığım ml modeline input olarak vericem.


In [ ]:
df = df.dropna(subset=["Customer ID"])
df.isna().sum()

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"]) # Tarihi tanıtalım
df["TotalPrice"] = df["Quantity"] * df["Price"]       # Gerçek ciroyu bulalım

# Analiz tarihi (Verideki son günden 2 gün sonrası)
today_date = df["InvoiceDate"].max() + pd.Timedelta(days=2)

rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda date: (today_date - date.max()).days, # Recency (Gün farkı)
    "Invoice": lambda num: num.nunique(),                       # Frequency (Eşsiz Fatura Sayısı)
    "TotalPrice": lambda price: price.sum()                     # Monetary (Toplam Ciro)
})

rfm.columns = ["Recency", "Frequency", "Monetary"]

# Negatif tutarları temizle (İadeler yüzünden eksi bakiye olmasın)
rfm = rfm[rfm["Monetary"] > 0]

print(rfm.head())

In [ ]:
sc = MinMaxScaler((0, 1))
rfm_scaled = sc.fit_transform(rfm)

In [ ]:
wcss = []

for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    # inertia_ : Küme içi hata değerini verir (Düşük olması iyidir)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Dirsek Yöntemi (Elbow Method)')
plt.xlabel('Küme Sayısı (K)')
plt.ylabel('Hata (Inertia)')
plt.show()

# Çalışma mantığı :  
Biz müşterileri gruplara ayırıcaz bu sayılar ise kaç grup olmalı onun bize cevabını vericek. 1'den 10'a kadar tek tek gruplara ayırıyoruz ve hata payına bakıyoruz. Eğer gruplar çok sıkı ve düzgünse (kmeans.inertia_) sayı DÜŞÜK çıkar (İyi). Eğer grup çok dağınık, herkes birbirinden uzaksa sayı YÜKSEK çıkar (Kötü).

Örnek :
*   1 Grup denedim $\rightarrow$ Hata: 450 Çok yüksek, herkes tek torbada
*   2 Grup denedim $\rightarrow$ Hata: 150 (Bayağı düştü)

# Yorumum :
Kırılma anım matematiksel olarak en mantıklı seçimim ama işletmeci olarak düşünürsem iki farklı katogori kurmak sadık müşteriler ve olmayan müşteriler diye işime gelmez. Üç seçip Düşük", "Orta", "Yüksek" diye ayırmak daha mantıklı duruma göre dörde ayrılıp daha ince davranılabilinir.

In [ ]:
# 1. Önce K=3 Modeli
kmeans_3 = KMeans(n_clusters=3, random_state=42)
rfm["Segment_3lu"] = kmeans_3.fit_predict(rfm_scaled)

# 2. Sonra K=4 Modeli
kmeans_4 = KMeans(n_clusters=4, random_state=42)
rfm["Segment_4lu"] = kmeans_4.fit_predict(rfm_scaled)

# ==========================================
# KARŞILAŞTIRMA RINGS
# ==========================================

print("------ 3 KÜMELİ SONUÇ (ORTALAMALAR) ------")
print(rfm.groupby("Segment_3lu")[["Recency", "Frequency", "Monetary"]].mean())
print("\nKişi Sayıları:")
print(rfm["Segment_3lu"].value_counts())

print("\n" + "="*40 + "\n")

print("------ 4 KÜMELİ SONUÇ (ORTALAMALAR) ------")
print(rfm.groupby("Segment_4lu")[["Recency", "Frequency", "Monetary"]].mean())
print("\nKişi Sayıları:")
print(rfm["Segment_4lu"].value_counts())

# En iyilerin arasındaki keskinlik farkı
3'lü grup için :
*   en son 53 gün gelmiş
*   10 kez alışveriş yapmış
*   4100 dolar para bırakmış

4'lü grup için :
*   en son 38 gün gelmiş
*   11 kez alışveriş yapmış
*   4500 dolar para bırakmış

Yukardaki yorumlara bakrak rahatlıkla 3'lü grup daha mnatıklı denilebilinir.

# En Kötü arasındaki keskinlik farkı
3'lü grup için :
*   en son 600 gün gelmiş
*   2 kez alışveriş yapmış
*   520 dolar para bırakmış

4'lü grup için :
*   en son 620 gün gelmiş
*   2 kez alışveriş yapmış
*   480 dolar para bırakmış

Bunlarada baktıktan sonra şu yorumu rahatlıkla yapabiliriz. Biz dörtlü grup yaptığımız zaman orta grubu ikiye ayırıyoruz ve işletme buna gerek var mı sorusunu sorarak bu kararı verebilir.

In [ ]:
# Aksiyon
def aksiyon_sihirbazi(row):
    if row['Recency'] <= 53 and row['Frequency'] >= 10 and row['Monetary'] > 4100:
        return "VIP Hediye"

    elif row['Recency'] >= 600 and row['Frequency'] <= 2 and row['Monetary'] <= 520:
        return "Standart İndirim Maili"

    else:
        return "Sadakat Kartı Öner"

rfm["Segment"] = rfm.apply(aksiyon_sihirbazi, axis=1)

print(rfm["Segment"].value_counts())

In [ ]:
sns.countplot(x='Segment', data=rfm, hue='Segment', legend=False,
              palette={'VIP Hediye': 'gold',
                       'Sadakat Kartı Öner': 'teal',
                       'Standart İndirim Maili': 'crimson'})

plt.title("Müşteri Segmentlerinin Kişi Sayısı Dağılımı")
plt.xlabel("Segment")
plt.ylabel("Kişi Sayısı")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors = {'VIP Hediye': 'gold',
          'Sadakat Kartı Öner': 'teal',
          'Standart İndirim Maili': 'crimson'}

for segment, color in colors.items():
    subset = rfm[rfm['Segment'] == segment]
    ax.scatter(subset['Recency'], subset['Frequency'], subset['Monetary'],
               c=color, label=segment, s=50, alpha=0.6)

ax.set_xlabel('Recency (Gün)')
ax.set_ylabel('Frequency (Sıklık)')
ax.set_zlabel('Monetary (Para)')
ax.set_title('Müşteri Segmentlerinin 3D Konumu')
ax.legend()

plt.show()